In [ ]:
import pandas as pd
import xarray as xr
from load import load_global_tas, load_sherwood, load_zelinka
tas = load_global_tas().set_index("year")
tas.columns = tas.columns.str.split("_", expand=True)
tas = xr.DataArray(tas,dims=['year','model'],name='tas').rename({"model_level_0":"generation","model_level_1":"center","model_level_2":"scenario","model_level_3":"real"})
import json
import urllib
zelinka_url = "https://raw.githubusercontent.com/mzelinka/cmip56_forcing_feedback_ecs/master/cmip56_forcing_feedback_ecs.json"
data = urllib.request.urlopen(zelinka_url)
zelinka = json.load(data)
zelinka = {k: v for k, v in zelinka.items() if k in ["CMIP6","CMIP5"]}
zelinka = pd.json_normalize(zelinka, sep="_")
zelinka.columns = zelinka.columns.str.split("_", expand=True)
zelinka = zelinka.transpose().unstack()
zelinka.columns = zelinka.columns.droplevel(0)
zelinka = xr.Dataset(zelinka).rename({"dim_0":"model","dim_0_level_0":"generation","dim_0_level_1":"center","dim_0_level_2":"real"})
#ecs = load_zelinka()
sherwood = load_sherwood()


In [ ]:
import panel as pn
import holoviews as hv
import hvplot.xarray
pn.extension()
import numpy as np
refyears = 29

tas_filt = tas
#select_model = pn.widgets.Select(options={m:i for i,m in enumerate(tas_filt['model'].values)},value=0)

#tas.unstack('model').hvplot(x='year',by='center')


In [ ]:
refs_start = range(tas.year.values[0]+1,tas.year.values[-1]-refyears,10)
w_reference_period = pn.widgets.DiscreteSlider(name="reference period", 
                                               options={f"{m}-{m+refyears}":m for i,m in enumerate(refs_start)},
                                               value=1991)
w_generation  = pn.widgets.Select(name="generation",  disabled = False, 
                                  options=['all',*list(np.unique(tas_filt['generation'].values))],
                                  value = 'CMIP6')
w_center      = pn.widgets.Select(name="center",      disabled = False , 
                                  options=['all',*list(np.unique(tas_filt['center'].values))] )
w_scenario    = pn.widgets.Select(name="scenario",    disabled = False , 
                                  options=['all',*list(np.unique(tas_filt['scenario'].values))] )
w_realization = pn.widgets.Select(name="real", disabled = False , 
                                  options=['all','first',*list(np.unique(tas_filt['real'].values))] )


In [ ]:

#@pn.depends(w_generation,watch=True)
def update_center(gen):
    if gen != 'all':
        gen_sel = tas_filt.sel(generation=gen).dropna(dim='model',how='all')
        scen_gen = np.unique(gen_sel['center'].values)
        w_center.options = list(['all',*scen_gen])
        w_center.disabled = False
    else:
        w_center.disabled = True
        


In [ ]:
##@pn.depends(w_generation,watch=True)
def update_scenario(gen):
    if gen != 'all':
        gen_sel = tas_filt.sel(generation=gen).dropna(dim='model',how='all')
        scen_gen = np.unique(gen_sel['scenario'].values)
        w_scenario.options = list(['all',*scen_gen])
        w_scenario.disabled = False
    else:
        w_scenario.disabled = True
        


In [ ]:
#@pn.depends(w_generation,watch=True)
def update_realization(gen):
    if gen != 'all':
        gen_sel = tas_filt.sel(generation=gen).dropna(dim='model',how='all')
        scen_gen = np.unique(gen_sel['real'].values)
        w_realization.options = list(['all','first',*scen_gen])
        w_realization.disabled = False
    else:
        w_realization.disabled = True
        


In [ ]:
fut = slice(2071,2100)


def tas_select(gen='all',center='all',scenario='all',realization='all',refstart=1991):
    tas_sel = tas_filt
    tas_sel = tas_sel.sel(generation=gen) if gen != 'all' else tas_sel
    tas_sel = tas_sel.sel(center=center) if center != 'all' else tas_sel
    tas_sel = tas_sel.sel(scenario=scenario) if scenario != 'all' else tas_sel
    tas_sel = tas_sel.sel(real=realization) if realization not in ['all','first'] else tas_sel

    ref = tas_sel.sel(year=slice(refstart,refstart+refyears)).mean(dim="year")
    delta = tas_sel.sel(year=fut).mean(dim="year") - ref
    return delta

@pn.depends(gen=w_generation.value)
def plot_kde(gen):
    delta = tas_select(gen,center,scenario,realization,refstart)
    return pn.Row(pn.pane.DataFrame(delta.to_dataframe()),)

In [ ]:

w_threshold = pn.widgets.FloatSlider(name="threshold", value=1.5, start=0.0, end=6.0, step=0.5)
@pn.depends(w_generation,w_center,w_scenario,w_realization,w_reference_period,w_threshold)
def year_kde(gen,center,scenario,realization,refstart,threshold):
    tas_sel = tas_filt.sel(generation=(gen if gen != 'all' else slice(None)),
                            center=(center if center != 'all' else slice(None)),
                            scenario=(scenario if scenario != 'all' else slice(None)),
                            realization=(realization if realization not in ['all','first'] else slice(None))
                    )
    ref = tas_sel.sel(year=slice(refstart,refstart+refyears)).mean(dim="year")
    delta = tas_sel - ref
    delta = delta.where(delta > threshold,drop=True)
    deltayr = {}
    for model in delta.model.values:
        deltayr[model] = delta.sel(model=model).dropna(dim='year',how='all').year.min()
    deltayr = pd.DataFrame(deltayr)
    return pn.Row(pn.pane.DataFrame(deltayr,height=400),
                  deltayr.hvplot(xlabel='year when tas(fut)-tas(ref) > thresh',height=400))


In [ ]:

layout = pn.Column(pn.Row(w_generation),
                   pn.Row(w_reference_period,pn.panel(f"Future period: **{fut.start}-{fut.stop}**")),
                   plot_kde)
layout.servable()